In [79]:
import pickle
import os
import numpy as np

## Data splitting

In [80]:
with open('anishDataset.pkl','rb') as f:
    dataset = pickle.load(f)

In [81]:
x = dataset['features']
np.array(x).shape

(120, 20, 64, 64, 3)

In [82]:
y = dataset['labels']
y = []
for label in dataset['labels']:
    if(label == 'correct'):
        y.append(1)
    if(label == 'incorrect'):
        y.append(0)

In [83]:
# dataset['labels'] = y

In [84]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(np.array(dataset['features']),np.array(dataset['labels']), test_size=0.25, random_state=100)

## Training

In [85]:
from keras.layers import Dense, ConvLSTM2D, MaxPooling3D, TimeDistributed, Dropout, Flatten
from keras.models import Sequential
from keras.callbacks import EarlyStopping

In [86]:
def create_model():
    model = Sequential()

    model.add(ConvLSTM2D(filters=4, kernel_size=(3,3), activation='tanh', data_format='channels_last', recurrent_dropout=0.2, return_sequences=True,
                         input_shape = (20,64,64,3)))
    model.add(MaxPooling3D(pool_size=(1,2,2),padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(ConvLSTM2D(filters=8, kernel_size=(3,3), activation='tanh', data_format='channels_last', recurrent_dropout=0.2, return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1,2,2),padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(ConvLSTM2D(filters=14, kernel_size=(3,3), activation='tanh', data_format='channels_last', recurrent_dropout=0.2, return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1,2,2),padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(ConvLSTM2D(filters=14, kernel_size=(3,3), activation='tanh', data_format='channels_last', recurrent_dropout=0.2, return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1,2,2),padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(ConvLSTM2D(filters=16, kernel_size=(3,3), activation='tanh', data_format='channels_last', recurrent_dropout=0.2, return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1,2,2),padding='same', data_format='channels_last'))
    
    model.add(Flatten())

    model.add(Dense(2, activation='softmax'))

    model.summary()
    return model

In [87]:
model = create_model()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d_30 (ConvLSTM2D)  (None, 20, 62, 62, 4)    1024      
                                                                 
 max_pooling3d_30 (MaxPoolin  (None, 20, 31, 31, 4)    0         
 g3D)                                                            
                                                                 
 time_distributed_24 (TimeDi  (None, 20, 31, 31, 4)    0         
 stributed)                                                      
                                                                 
 conv_lstm2d_31 (ConvLSTM2D)  (None, 20, 29, 29, 8)    3488      
                                                                 
 max_pooling3d_31 (MaxPoolin  (None, 20, 15, 15, 8)    0         
 g3D)                                                            
                                                      

In [88]:
from keras.callbacks import EarlyStopping, ModelCheckpoint


# Define Early Stopping Callback
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)

# Define ModelCheckpoint Callback to save the model at the end of training
model_checkpoint_callback = ModelCheckpoint(filepath='retrain_best_model.h5', save_best_only=True, save_weights_only=False, monitor='val_loss', mode='min')

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

# Train the model with validation data and callbacks
model.fit(xtrain, ytrain, epochs=50, batch_size=4, shuffle=True, callbacks=[early_stopping_callback, model_checkpoint_callback], validation_split=0.2)




Epoch 1/50
18/18 [==============================] - 17s 518ms/step - loss: 0.6943 - accuracy: 0.5139 - val_loss: 0.6924 - val_accuracy: 0.5000
Epoch 2/50
18/18 [==============================] - 8s 469ms/step - loss: 0.6922 - accuracy: 0.5556 - val_loss: 0.6905 - val_accuracy: 0.7222
Epoch 3/50
18/18 [==============================] - 8s 448ms/step - loss: 0.6952 - accuracy: 0.6111 - val_loss: 0.6775 - val_accuracy: 0.6111
Epoch 4/50
18/18 [==============================] - 8s 459ms/step - loss: 0.6822 - accuracy: 0.6806 - val_loss: 0.6634 - val_accuracy: 0.6667
Epoch 5/50
18/18 [==============================] - 8s 459ms/step - loss: 0.6073 - accuracy: 0.7083 - val_loss: 0.7129 - val_accuracy: 0.6667
Epoch 6/50
18/18 [==============================] - 8s 455ms/step - loss: 0.5768 - accuracy: 0.6806 - val_loss: 0.7383 - val_accuracy: 0.6667
Epoch 7/50
18/18 [==============================] - 8s 468ms/step - loss: 0.5297 - accuracy: 0.6944 - val_loss: 0.6086 - val_accuracy: 0.7222
Epoch

KeyboardInterrupt: 

In [95]:
from keras.models import load_model

# Load the best model saved during training
best_model = load_model('retrain_best_model.h5')

In [96]:
ypred = best_model.predict(xtest)

1/1 [==============================] - 1s 1s/step


In [97]:
# Define the threshold
threshold = 0.5

# Convert sigmoid outputs to binary values
ypred = np.argmax(ypred,axis=1)
ypred

array([1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 1, 0, 1], dtype=int64)

In [98]:
from sklearn.metrics import *

confusion_matrix(ytest,ypred)

array([[7, 7],
       [8, 8]], dtype=int64)